# Install RDkit in Colab

In [ ]:
import os
import subprocess
import sys

IN_COLAB = 'google.colab' in sys.modules

def run_cmd(cmd):
    print(f'Output of "{cmd}":')
    print(subprocess.run(cmd, stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8'))
    
def run_cmd_list(cmd_list):
    [run_cmd(cmd) for cmd in cmd_list]

def pip_install(module_list):
    run_cmd_list([f'pip install  --upgrade --force-reinstall {mod}' for mod in module_list])

def install_rdkit():
    cmd_list = [
        "wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh",
        "chmod +x Miniconda3-latest-Linux-x86_64.sh",
        "bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local",
        "conda install -q -y -c conda-forge rdkit",
        "rm -rf Miniconda3-latest-Linux-x86_64.sh"]
    print('Installing rdkit\n\n')
    run_cmd_list(cmd_list)
    custom_path = '/usr/local/lib/python3.7/site-packages/'
    print(f'Do not forget to append "{custom_path}"" to sys.path')
    
if IN_COLAB:
    SRC_DIR = '.'
    run_cmd('pip unistall tensorflow')
    pip_install(['tensorflow', 'tqdm','git+https://github.com/GPflow/GPflow.git@develop#egg=gpflow'])
    install_rdkit()    
else:
    SRC_DIR = '..'

print('Using colab? {}, using root directory "{}"'.format(IN_COLAB, SRC_DIR))

## Append to path

In [ ]:
import sys
import os
if 'google.colab' in sys.modules:
    SRC_DIR='.'
    sys.path.append('/usr/local/lib/python3.7/site-packages/')

## Libraries

In [ ]:
# utils
from collections import OrderedDict
from tqdm.autonotebook import tqdm
# scientific python
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import sklearn.ensemble
import scipy.stats

import tensorflow as tf
import gpflow
from gpflow.utilities import print_summary

# rdkit stuff
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole
print(f'rdkit : {rdkit.__version__}')
print(f' tf   : {tf.__version__}')
print(f'gpflow: {gpflow.__version__}')
tqdm.pandas()

In [ ]:
def rf_feature_mask(x, y, k):
   rf = sklearn.ensemble.RandomForestRegressor()
   rf.fit(x,y)
   indices = (-rf.feature_importances_).argsort()[:k]
   mask = np.zeros_like(rf.feature_importances_)
   mask[indices] = 1.0
   return mask.astype(bool)

class IdentityTransformer(sklearn.preprocessing.FunctionTransformer):
    
    def __init__(self):
        super().__init__(lambda x:x, lambda x:x)

class GPFeature():
    
    def __init__(self, label, x, y, n_features=None, preproc=None):
        self.label = label
        n_features = n_features or x.shape[-1]
        x_feats = x.shape[-1]
        if n_features > x_feats:
            raise ValueError(f'n_features={n_features} larger than x.shape[-1] {x_feats}')
        elif n_features < x_feats:
            mask = rf_feature_mask(x, y, n_features)
        else:
            mask = np.ones(x_feats).astype(bool)
        self.mask = mask
        preproc = preproc or IdentityTransformer()
        self.preproc = preproc.fit(x[:,self.mask])
        self.ndim = sum(self.mask)
        self.active_dims = np.arange(self.ndim)
    
    def transform(self, x):
        return self.preproc.transform(x[:,self.mask]).astype(np.float64)
    
def evaluate_result(y_true, y_pred, fold=None):
    result = OrderedDict()
    result['r2'] = sklearn.metrics.r2_score(y_true,y_pred)
    result['rmse'] = np.sqrt(sklearn.metrics.mean_squared_error(y_true,y_pred))
    result['mae'] = np.sqrt(sklearn.metrics.mean_absolute_error(y_true,y_pred))
    result['r'] = scipy.stats.pearsonr(y_test.ravel(),y_pred.ravel())[0]
    if fold:
        result['cv'] = fold
    return result


# Task and data: predict Melting Point 💧


## Let's load some molecular data with 🐼s

In [ ]:
!wget https://web.alcf.anl.gov/~vama/melting_point_data/table_47k.cvs

In [ ]:
df = pd.read_csv('table_47k.cvs')
print(df.shape)
print(df.columns)
smiles_column='smiles'
df['mol'] = df[smiles_column].apply(Chem.MolFromSmiles)
target='Tm'
df.head()

In [ ]:
getonly=['NHOHCount', 'NOCount', 'NumAliphaticCarbocycles',
                           'NumAliphaticHeterocycles', 'NumAliphaticRings',
                           'NumAromaticCarbocycles', 'NumAromaticHeterocycles',
                           'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 
                           'NumHeteroatoms', 'NumRadicalElectrons', 'NumRotatableBonds',
                           'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 
                           'NumSaturatedRings', 'NumValenceElectrons',
'qed','TPSA', 'MolMR','BalabanJ', 'BertzCT',
'fr_Al_OH', 'fr_Al_OH_noTert', 'fr_ArN', 'fr_Ar_COO',
                            'fr_Ar_N', 'fr_Ar_NH', 'fr_Ar_OH', 'fr_COO', 'fr_COO2', 
                            'fr_C_O', 'fr_C_O_noCOO', 'fr_C_S', 'fr_HOCCN', 'fr_Imine',
                            'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_N_O', 'fr_Ndealkylation1',
                            'fr_Ndealkylation2', 'fr_Nhpyrrole', 'fr_SH', 'fr_aldehyde',
                            'fr_alkyl_carbamate', 'fr_alkyl_halide', 'fr_allylic_oxid', 'fr_amide',
                            'fr_amidine', 'fr_aniline', 'fr_aryl_methyl', 'fr_azide', 'fr_azo',
                            'fr_barbitur', 'fr_benzene', 'fr_benzodiazepine', 'fr_bicyclic',
                            'fr_diazo', 'fr_dihydropyridine', 'fr_epoxide', 'fr_ester', 'fr_ether',
                            'fr_furan', 'fr_guanido', 'fr_halogen', 'fr_hdrzine', 'fr_hdrzone', 
                            'fr_imidazole', 'fr_imide', 'fr_isocyan', 'fr_isothiocyan', 'fr_ketone',
                            'fr_ketone_Topliss', 'fr_lactam', 'fr_lactone', 'fr_methoxy', 'fr_morpholine',
                            'fr_nitrile', 'fr_nitro', 'fr_nitro_arom', 'fr_nitro_arom_nonortho', 
                            'fr_nitroso', 'fr_oxazole', 'fr_oxime', 'fr_para_hydroxylation', 
                            'fr_phenol', 'fr_phenol_noOrthoHbond', 'fr_phos_acid', 'fr_phos_ester', 
                            'fr_piperdine', 'fr_piperzine', 'fr_priamide', 'fr_prisulfonamd',
                            'fr_pyridine', 'fr_quatN', 'fr_sulfide', 'fr_sulfonamd', 'fr_sulfone',
                            'fr_term_acetylene', 'fr_tetrazole', 'fr_thiazole', 'fr_thiocyan', 
                            'fr_thiophene', 'fr_unbrch_alkane', 'fr_urea',
                            'MolWt','MolLogP']

In [ ]:
import inspect
import rdkit.Chem.Descriptors as Descriptors


calc_props = OrderedDict(inspect.getmembers(Descriptors, inspect.isfunction))
for key in list(calc_props.keys()):
    if key.startswith('_'):
        del calc_props[key]
def calc_all(df,calc_props,smiles_col='smiles'):
    #df['mol'] = df[smiles_col].apply(Chem.MolFromSmiles)
    for key,val in tqdm(calc_props.items()):
        if key in getonly:
           df[key] = df['mol'].apply(val)
    return df
print('Found {} molecular descriptors in RDKIT'.format(len(calc_props)))
#calc_props

In [ ]:
calc_all(df,calc_props)
print(df.shape)
df.head(1)


## Train/test splits

In [ ]:
indices=df.index.tolist()
train_index,test_index = sklearn.model_selection.train_test_split(indices, test_size=.20)
print(len(train_index),len(test_index))

## Target

In [ ]:
target='Tm'
y_true = df[target].values.reshape(-1,1).astype(np.float64)
y_preproc = sklearn.preprocessing.StandardScaler()
y_train = y_preproc.fit_transform(y_true[train_index])
y_test = y_true[test_index]
print(y_train.shape, y_test.shape)

## Fingerprint features

In [ ]:
fp = np.vstack(df['mol'].apply(lambda m: np.array(AllChem.GetMorganFingerprintAsBitVect(m,3,nBits=2048))).values)
print(fp.shape)
fp_feat = GPFeature('fp', fp[train_index], y_train, 200)
fp_train = fp_feat.transform(fp[train_index]) 
fp_test = fp_feat.transform(fp[test_index]) 
print(fp_train.shape,fp_test.shape)

## chemoinformatic features

In [ ]:
other_features = getonly
x_tmp = df[other_features].values
chem_feat = GPFeature('chemoinformatic', x_tmp[train_index],y_train, 4, sklearn.preprocessing.StandardScaler())
chem_train = chem_feat.transform(x_tmp[train_index])
chem_test = chem_feat.transform(x_tmp[test_index])
print(chem_train.shape,chem_test.shape)

## Stack features into one numpy array

In [ ]:
x_train = np.hstack((fp_train,chem_train))
x_test = np.hstack((fp_test,chem_test))
print(x_train.shape,x_test.shape)

# GP regression
## Build kernel

In [ ]:
last_dim = 0
fp_kernel = gpflow.kernels.RBF(active_dims=fp_feat.active_dims+last_dim)
last_dim  = fp_kernel.active_dims[-1]+1
chem_kernel = gpflow.kernels.RBF(active_dims=chem_feat.active_dims+last_dim)
kernel = fp_kernel + chem_kernel
print_summary(kernel)

## Specify the model

In [ ]:
model = gpflow.models.GPR(data=(x_train, y_train), kernel=kernel, mean_function=None)

## Optimize model

In [ ]:
opt = gpflow.optimizers.Scipy()

def objective():
    return - model.log_marginal_likelihood()

opt_logs = opt.minimize(objective,
                        model.trainable_variables,
                        options=dict(maxiter=500))
print_summary(model)

## Results

In [ ]:
y_pred, y_std  = model.predict_y(x_test)
y_pred = y_preproc.inverse_transform(y_pred.numpy())
results = [evaluate_result(y_test, y_pred)]
pd.DataFrame(results)